# Optimized Resource Aware Pruning (ORAP)

In this session you will see:
- Resource demand calculation of decision trees
- The general concept of global optimization 
- Optimal solution search using dynamic programming
- ORAP in action
- Comparison of ORAP with CCP